In [8]:
from pydrake.all import ModelVisualizer, Simulator, StartMeshcat
from manipulation import ConfigureParser, running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation

from typing import Callable

import numpy as np
from pydrake.all import (
    BasicVector,
    Box,
    ConstantVectorSource,
    Context,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MathematicalProgram,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    SnoptSolver,
    ge,
    le,
    Solve,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_differential_ik import TestDifferentialIK
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import RenderDiagram

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
class DifferentialIKSystem(LeafSystem):
    """Wrapper system for Differential IK.
    @param plant MultibodyPlant of the simulated plant.
    @param diffik_fun function object that handles diffik. Must have the signature
           diffik_fun(J_G, V_G_desired, q_now, v_now, X_now)
    """

    def __init__(self, plant: MultibodyPlant, diffik_fun: Callable) -> None:
        self.val = True
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()
        self._diffik_fun = diffik_fun

        self.DeclareVectorInputPort("spatial_velocity", 6)
        self.DeclareVectorInputPort("iiwa.position_measured", 7)
        self.DeclareVectorInputPort("iiwa.velocity_measured", 7)
        self.DeclareVectorOutputPort("iiwa_velocity_command", 7, self.CalcOutput)

    def CalcOutput(self, context: Context, output: BasicVector) -> None:
        V_G_desired = self.get_input_port(0).Eval(context)
        q_now = self.get_input_port(1).Eval(context)
        v_now = self.get_input_port(2).Eval(context)

        self._plant.SetPositions(self._plant_context, self._iiwa, q_now)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kQDot,
            self._G,
            [0, 0, 0],
            self._W,
            self._W,
        )
        J_G = J_G[:, 0:7]  # Ignore gripper terms

        X_now = self._plant.CalcRelativeTransform(self._plant_context, self._W, self._G)
        p_now = X_now.translation()

        # You will be defining different versions of the diffik_fn in this problem.
        v = self._diffik_fun(J_G, V_G_desired, q_now, v_now, p_now)
        output.SetFromVector(v)


def BuildAndSimulate(
    diffik_fun: Callable,
    V_d: np.ndarray,
    duration: float,
    plot_system_diagram: bool = False,
) -> None:
    builder = DiagramBuilder()

    scenario_data = """
        directives:
        - add_model:
            name: iiwa
            file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
            default_joint_positions:
                iiwa_joint_1: [-1.57]
                iiwa_joint_2: [0.1]
                iiwa_joint_3: [0]
                iiwa_joint_4: [-1.2]
                iiwa_joint_5: [0]
                iiwa_joint_6: [1.6]
                iiwa_joint_7: [0]

        - add_model:
            name: wsg
            file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
            
        - add_weld:
            parent: world
            child:  iiwa::iiwa_link_0

        - add_weld:
            parent: iiwa::iiwa_link_7
            child: wsg::body
            X_PC:
                translation: [0, 0, 0.09]
                rotation: !Rpy { deg: [90, 0, 90]}

        - add_model:
            name: hoop_model
            file: file:///workspaces/6.4210-final-project/sdfs/basketball_hoop.sdf

        - add_weld:
            parent: world
            child: hoop_model::base_link_hoop  
            X_PC:
                translation: [5, 0, 3.048]
                rotation: !Rpy { deg: [0, 0, 180] }

        - add_model:
            name: ball
            file: package://drake_models/manipulation_station/sphere.sdf

        - add_weld:
            parent: world
            child: ball::base_link
            X_PC:
                translation: [0.5, 0, 0]
                rotation: !Rpy { deg: [0, 0, 0] }

        model_drivers:
            iiwa: !IiwaDriver
                hand_model_name: wsg
            wsg: !SchunkWsgDriver {}
    """

    scenario = LoadScenario(data=scenario_data)
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
    plant = station.GetSubsystemByName("plant")

    controller = builder.AddSystem(DifferentialIKSystem(plant, diffik_fun))
    integrator = builder.AddSystem(Integrator(7))
    desired_vel = builder.AddSystem(ConstantVectorSource(V_d))

    builder.Connect(controller.get_output_port(), integrator.get_input_port())
    builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
    builder.Connect(
        station.GetOutputPort("iiwa.position_measured"),
        controller.get_input_port(1),
    )
    builder.Connect(
        station.GetOutputPort("iiwa.velocity_estimated"),
        controller.get_input_port(2),
    )
    builder.Connect(desired_vel.get_output_port(), controller.get_input_port(0))

    diagram = builder.Build()
    diagram.set_name("diagram")
    if running_as_notebook and plot_system_diagram:
        RenderDiagram(diagram)

    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyContextFromRoot(context)
    station.GetInputPort("iiwa.torque").FixValue(station_context, np.zeros((7, 1)))
    station.GetInputPort("wsg.position").FixValue(station_context, [0.1])

    integrator.set_integral_value(
        integrator.GetMyMutableContextFromRoot(context),
        plant.GetPositions(
            plant.GetMyContextFromRoot(context),
            plant.GetModelInstanceByName("iiwa"),
        ),
    )

    meshcat.StartRecording()
    simulator.AdvanceTo(duration)
    meshcat.PublishRecording()

In [4]:
def DiffIKQP(
    J_G: np.ndarray,
    V_G_desired: np.ndarray,
    q_now: np.ndarray,
    v_now: np.ndarray,
    p_now: np.ndarray,
) -> np.ndarray:
    prog = MathematicalProgram()
    v = prog.NewContinuousVariables(7, "v")
    v_max = 3.0  # do not modify

    # TODO: Add cost and constraints to prog here.
    prog.AddCost((J_G @ v - V_G_desired).dot(J_G @ v - V_G_desired))
    prog.AddConstraint(le(v, v_max))
    prog.AddConstraint(ge(v, -v_max))

    solver = SnoptSolver()
    result = solver.Solve(prog)
    if not (result.is_success()):
        raise ValueError("Could not find the optimal solution.")

    v_solution = result.GetSolution(v)

    return v_solution

In [ ]:
g = [0, 0, -9.81]

def calculate_ball_release_pose(X_H, X_B, entry_angle, s_hit, T_bounds):
    prog = MathematicalProgram()
    p_rel = prog.NewContinuousVariables(3, "p_rel")
    v_rel = prog.NewContinuousVariables(3, "v_rel")
    T = prog.NewContinuousVariables(1, "T")[0]
    
    # Get entry direction for a given angle
    p_hit = X_H.translation()
    R_WH = X_H.rotation()
    z_H = R_WH[2]
    
    down = -z_H / np.linalg.norm(z_H)
    
    world_up = np.array([0., 0., 1.])
    horizontal = np.cross(down, world_up)
    
    v_dir = np.cos(entry_angle) * down + np.sin(entry_angle) * horizontal
    v_dir = v_dir / np.linalg.norm(v_dir)
    
    v_hit = s_hit * v_dir
    
    # projectile constraints
    T_min, T_max = T_bounds
    prog.AddBoundingBoxConstraint(T_min, T_max, T)
    
    for i in range(3):
        prog.AddConstraint(v_rel[i] + g[i] * T == v_hit[i])
        prog.AddConstraint(p_rel[i] + v_rel[i] * T + 0.5 * g[i] * T**2 == p_hit[i])
        
    # Solve for v_rel
    prog.AddQuadraticCost(v_rel @ v_rel)
    result = Solve(prog)
    
    p_rel_val = result.GetSolution(p_rel)
    v_rel_val = result.GetSolution(v_rel)
    T_val = result.GetSolution(T)

    return p_rel_val, v_rel_val, T_val
    
    
    
    
    

In [ ]:
def make_grasp_poses():
    # ball center in world frame
    p_WB = np.array([0.5, 0.0, 0.0])  

    # Approach pose: slightly above ball
    p_WG_approach = p_WB + np.array([0.0, 0.0, 0.15])

    # Orientation: gripper z-axis down
    R_WG = RollPitchYaw(np.pi, 0.0, 0.0).ToRotationMatrix()

    X_WG_approach = RigidTransform(R_WG, p_WG_approach)
    X_WG_grasp    = RigidTransform(R_WG, p_WB)   # right at ball center

    return X_WG_approach, X_WG_grasp



def solve_ik_for_pose(plant, X_WG_desired, q_seed):
    ik = InverseKinematics(plant, plant.CreateDefaultContext())
    q = ik.q()

    G = plant.GetBodyByName("body").body_frame()
    W = plant.world_frame()
    ik.AddPositionConstraint(G, [0, 0, 0], W, X_WG_desired.translation(), X_WG_desired.translation())
    ik.AddOrientationConstraint(G, RigidTransform(X_WG_desired.rotation(), [0,0,0]), W,
                                RigidTransform(X_WG_desired.rotation(), [0,0,0]),
                                0.01)  # small angular tolerance

    ik.prog().AddQuadraticCost((q - q_seed).dot(q - q_seed))
    result = Solve(ik.prog())
    if not result.is_success():
        raise RuntimeError("IK failed")
    return result.GetSolution(q)

